**Producer Analysis Project**  

In [1]:
##Load in the LyricsGenius package , https://lyricsgenius.readthedocs.io/en/master/how_it_works.html
# https://pypi.org/project/lyricsgenius/
# https://www.johnwmillr.com/scraping-genius-lyrics/
# https://lightrun.com/answers/johnwmillr-lyricsgenius-timeout-raised-on-search
from lyricsgenius import Genius

##Set values for Genius API keys
clientID = "44n0C3tTGnF7wem8qFlrx4Lj_3lN7YxKaU6dNKGnOWTRI4Qicv1SeaDIDwXLiWJa"
clientSecret = "vFrKq-wzDNG3qx1_J6MJ8iuqPCH10SUzI0dpeFszbGC6rFT-wlGGr57zoTUoIMp0KsWY4ZF7KjJToBuR2L0NYw"
accessToken = "VPA-QksK9J14_qmP0MpyDk1DvAv5hx_xCJ-eIh8lD_hkeoDu-DS5DsLeJf28ix5x"

**Pull in the data for Metro Boomin**

In [42]:
#Make the genius API object
genius = Genius(accessToken)
genius.skip_non_songs = True
genius.include_features = True
genius.excluded_terms = ["(Instrumental)","(Instrumental - Bonus)"]
genius.remove_section_headers = True
genius.timeout = 60
genius.retries = 5

In [43]:
#Search for songs produced by Metro Boomin
MetroBoomin = genius.search_artist("Metro Boomin", sort="title")

Searching for songs by Metro Boomin...

Song 1: "10AM/Save the World"
"10AM / Save the World (Instrumental)" is not valid. Skipping.
Song 2: "10 Freaky Girls"
"10 Freaky Girls (Instrumental)" is not valid. Skipping.
"1st Of Da Month" is not valid. Skipping.
"25K" is not valid. Skipping.
"All The Money (Instrumental - Bonus)" is not valid. Skipping.
"All The Way Live (Spider-Man: Across the Spider-Verse - Instrumental)" is not valid. Skipping.
"Am I Dreaming (Instrumental)" is not valid. Skipping.
"Annihilate (Spider-Man: Across the Spider-Verse - Instrumental)" is not valid. Skipping.
"Another Body" is not valid. Skipping.
Song 3: "Around Me"
Song 4: "Around Me (ChoppedNotSlopped)"
"Around Me (Instrumental)" is not valid. Skipping.
"BBL*" is not valid. Skipping.
"Big Mad" is not valid. Skipping.
"Bitch Ass" is not valid. Skipping.
"Bitch Boy*" is not valid. Skipping.
Song 5: "Blue Pill"
Song 6: "Blue Pill (Demo)"
Song 7: "Borrowed Love"
"Borrowed Love (Instrumental)" is not valid. Skip

In [1]:
##Create and save a dataframe for Metro Boomin Songs'
import pandas as pd
MetroData = pd.DataFrame()
artist = []
title = []
lyrics = []
producer = []

#Create data set
for song in MetroBoomin.songs:
    artist.append(song.artist)
    title.append(song.title_with_featured)
    # Split the string into lines (This is to remove "Contributors" from the lyrics string)
    lines = song.lyrics.splitlines()

    # Exclude the first line and join the remaining lines
    fixedLyrics = '\n'.join(lines[1:])
    fixedLyrics = fixedLyrics.replace("\n", " ")
    
    lyrics.append(fixedLyrics)
    producer.append("Metro Boomin")

#Put the data frame together
MetroData["Artist"] = artist
MetroData["Song Title"] = title
MetroData["Producer"] = producer
MetroData["Lyrics"] = lyrics

#Print the results
print(MetroData)

NameError: name 'MetroBoomin' is not defined

In [95]:
#Save the metroboomin data frame
MetroData.to_csv('MetroBoomin.csv', index=True)

**Making a function to make pulling Data easier**

In [4]:
#So I can leave this to work overnight I am making it work with a list of artist names rather than just a single one.
from lyricsgenius import Genius


def pullAndSave(nameList, geniusObject):
    import pandas as pd
    #Iterate through the artists
    for person in nameList:
        saveName = person
        person = geniusObject.search_artist(person, sort="title")
        #Iterate through the songs scrapped for the artist and prepare the dataframe to store the song info.
        PersonData = pd.DataFrame()
        artist = []
        title = []
        lyrics = []
        album = []
        producerColumn = []
        for song in person.songs:
            songDict = song.to_dict()
            producerInfo = songDict['producer_artists']
            #Get the producers for the song
            individual = ""
            count = 0
            for json in producerInfo:
                if count == 0:
                    individual += json["name"]
                    count += 1
                else:
                    individual += " & " + json["name"]
            #Append basic song info to lists, need to do a try and except for album names.
            try:
                albumTitle = songDict["album"]["name"]
            except:
                print("Album title was not found")
                albumTitle = "N/A"
            print(albumTitle)
            album.append(albumTitle)
            artist.append(song.artist)
            title.append(song.title_with_featured)
            # Split the string into lines (This is to remove "Contributors" from the lyrics string)
            lines = song.lyrics.splitlines()
            # Exclude the first line which is contributors and join the remaining lines
            fixedLyrics = '\n'.join(lines[1:])
            fixedLyrics = fixedLyrics.replace("\n", " ")
            lyrics.append(fixedLyrics)
            if individual == '':
                producerColumn.append("NA")
            else:
                producerColumn.append(individual)
        #Finally, Put the data frame together
        PersonData["Artist"] = artist
        PersonData["Album"] = album
        PersonData["Song Title"] = title
        PersonData["Producer"] = producerColumn
        PersonData["Lyrics"] = lyrics
        #Save the dataframe to a csv file:
        fileName = saveName + ".csv"
        fileName = fileName.replace(" " , "")
        PersonData.to_csv(fileName, index=True)

**Test the Function:**

In [5]:
##Set values for Genius API keys
from lyricsgenius import Genius
clientID = "44n0C3tTGnF7wem8qFlrx4Lj_3lN7YxKaU6dNKGnOWTRI4Qicv1SeaDIDwXLiWJa"
clientSecret = "vFrKq-wzDNG3qx1_J6MJ8iuqPCH10SUzI0dpeFszbGC6rFT-wlGGr57zoTUoIMp0KsWY4ZF7KjJToBuR2L0NYw"
accessToken = "VPA-QksK9J14_qmP0MpyDk1DvAv5hx_xCJ-eIh8lD_hkeoDu-DS5DsLeJf28ix5x"

#Make the geniusAPI object to go into the function
genius = Genius(accessToken)
genius.skip_non_songs = True
genius.include_features = True
genius.excluded_terms = ["(Instrumental)","(Instrumental - Bonus)"]
genius.remove_section_headers = True
genius.timeout = 60
genius.retries = 5


artistList = ["Gucci Mane" , "Taylor Swift" , "Lana Del Rey", "Mike Dean", "Yung Lean", "Drake", "Avril Lavigne", "Tyler, The Creator", "Playboi Carti", "Ellie Goulding","Kanye West","Chief Keef","A$AP Rocky", "Ed Sheeran", "LUCKI", "Kid Cudi","Katy Perry","Metro Boomin"]
pullAndSave(artistList,genius)

Searching for songs by Gucci Mane...

Song 1: "06 Gucci"
Song 2: "100 Bails Of Kush"
Song 3: "100 Grand"
"100 Racc$" is not valid. Skipping.
Song 4: "1017 Brick Squad (The Movement)"
Song 5: "1017 Freestyle"
Song 6: "1017 Mafia"
Song 7: "12"
Song 8: "12 Days of Christmas"
Song 9: "15 Minutes Past the Diamond"
Song 10: "16 Fever"
Song 11: "1982"
Song 12: "1 Foot Forward"
Song 13: "1st Day out Tha Feds"
Song 14: "2010 Style"
Song 15: "24 Hours"
Song 16: "26  Inches"
Song 17: "2 Deep"
"2 Dope Boy Knot" is not valid. Skipping.
Song 18: "2 Screws Loose"
Song 19: "2 Seater"
Song 20: "2 Times"
Song 21: "30 Bricks"
Song 22: "30 Years 30 Million"
Song 23: "3 Extra"
Song 24: "3rd Quarter"
Song 25: "4Real"
Song 26: "50 Floors Up"
Song 27: "5 Million Intro"
Song 28: "5 Oâ€™Clock"
Song 29: "#6 (Intro)"
Song 30: "745"
Song 31: "911 Emergency"
Song 32: "Activist"
Song 33: "Act Up"
Song 34: "Addicted"
Song 35: "Add It Up"
Song 36: "A Failure"
Song 37: "Aggressive"
Song 38: "Aggressive (Remix)"
Song 39

Song 263: "Crackpot"
Song 264: "Crash"
Song 265: "Crazy"
Song 266: "Crazy Things"
Song 267: "Crowd In Rage"
Song 268: "Crush"
"Culinary Art School" is not valid. Skipping.
Song 269: "Curve"
Song 270: "Cut It"
Song 271: "Cutters"
Song 272: "Cuttin Off Fingaz"
Song 273: "Câ€™Yeah"
Song 274: "Da Beginning"
Song 275: "Da Gun"
Song 276: "Damn Shawty"
"Damn Shawty (Instrumental)" is not valid. Skipping.
Song 277: "Dance with the Devil"
Song 278: "Dangerous"
Song 279: "Dangerâ€™s Not A Stranger"
Song 280: "Danny Glover"
Song 281: "Darker"
Song 282: "Dat Boi Cold"
"Dat Girl" is not valid. Skipping.
Song 283: "Dats My Life"
Song 284: "Dboy Style"
Song 285: "Dead Broke"
Song 286: "Dead End"
Song 287: "Dead Man"
Song 288: "Dead People"
Song 289: "Decapitated"
Song 290: "Deuces (Remix)"
Song 291: "Devils in My Section"
Song 292: "Diamond Chains"
Song 293: "Diamonds"
Song 294: "Dickriders"
Song 295: "D.I.G. (Dipped in Gold)"
Song 296: "Directorâ€™s Chair"
Song 297: "Dirt Cheap"
Song 298: "Dirt Chea

"Gucci Ninja Assassins 2: In the Metal Garden of Fire" is not valid. Skipping.
Song 510: "Gucci On the Rise"
Song 511: "Gucci Please"
Song 512: "Gucci Rag Top"
Song 513: "Gucci Speaks 2 (Burrprint 3-D)"
Song 514: "Gucci Speaks 2 (Writing on the Wall)"
Song 515: "Gucci Speaks 3 (Burrprint 3-D)"
Song 516: "Gucci Speaks (Burrprint 3-D)"
Song 517: "Gucci Speaks (Jewelry Selection)"
Song 518: "Gucci Speaks (Writing on the Wall)"
Song 519: "Gucci Talk"
Song 520: "Gucci Talk Intro"
Song 521: "Gucci Talk Outro"
Song 522: "Gucci Talk (Part 2)"
Song 523: "Gucci Talk (WOTW2)"
Song 524: "Gucci Terintino"
Song 525: "Gucci the Eskimo"
Song 526: "Gucci Time"
Song 527: "Gucci Time Remix"
Song 528: "Guest List"
Song 529: "Guilty"
Song 530: "Guinea Pig"
Song 531: "Guitar (Freestyle)"
Song 532: "Gunnin"
Song 533: "Guwop Home"
Song 534: "GuWop Nigga (Gucci & Trinidad)"
Song 535: "Gwap"
Song 536: "Gwops"
Song 537: "Gym"
Song 538: "Gymnast"
Song 539: "Half"
"Ham on â€™Em" is not valid. Skipping.
Song 540: "

Song 762: "Lights Off"
Song 763: "Lights On Lights Out"
Song 764: "Like 34 & 8"
Song 765: "Like I Used 2"
Song 766: "Like Mine"
Song 767: "Like the Gambinos"
Song 768: "Like the Grinch"
Song 769: "Lil Dudes"
Song 770: "Lil Friend"
Song 771: "Lil Kim Speaks"
Song 772: "Lil Story"
Song 773: "Little Friend"
Song 774: "Live At The Red Carpet"
Song 775: "Live from the Fulton County Jail Gucci Mane Speaks (Outro)"
Song 776: "Live in a TV"
Song 777: "Loaded"
"Loaded (1017 Remix)" is not valid. Skipping.
Song 778: "Lodi Dodi"
Song 779: "Long Live Dolph"
Song 780: "Long Money"
Song 781: "Long Time"
Song 782: "Long Way"
Song 783: "Look At Me Now"
Song 784: "Look Like a Lick"
Song 785: "Lord"
Song 786: "Loser"
Song 787: "Losin"
Song 788: "Loss 4 Wrdz"
"Lost My" is not valid. Skipping.
Song 789: "Lost My Mind"
Song 790: "Lost My Plug"
Song 791: "Lost Yâ€™all Mind"
Song 792: "Lots of Cash"
Song 793: "Lotz of Snow"
Song 794: "Loud"
Song 795: "Loud Pak"
"Loud Talk" is not valid. Skipping.
Song 796: "

Song 1020: "Pretty Girls"
Song 1021: "Prey"
Song 1022: "Product"
Song 1023: "Prom Night"
Song 1024: "Proud of You"
Song 1025: "Publicity Stunt"
Song 1026: "Pull Up On Ya"
Song 1027: "Pull Up To The Club"
Song 1028: "Punchanella (Gym)"
Song 1029: "Pushaman"
Song 1030: "Push Ups"
Song 1031: "Pussy Nigga"
Song 1032: "Pussy Nigga (Demo)"
"Pussy & Patron" is not valid. Skipping.
Song 1033: "Pussy Print"
Song 1034: "Pussy Rehab"
Song 1035: "Pussy Wet"
Song 1036: "Put on a Show"
Song 1037: "Put Some Wood In Her"
Song 1038: "Put Yourself In My Shoes"
Song 1039: "Pyrex Pot"
Song 1040: "Quiet"
Song 1041: "Rack City"
"Racks" is not valid. Skipping.
Song 1042: "Radical"
"Rain" is not valid. Skipping.
Song 1043: "Rainbow Colors"
Song 1044: "Rainin Money"
Song 1045: "Rain Man"
Song 1046: "Rain Shower"
Song 1047: "Ran Wit It"
"Rapid Fire" is not valid. Skipping.
Song 1048: "Rap Niggaz"
Song 1049: "Rather Be"
"Rat Traps" is not valid. Skipping.
Song 1050: "Real As They Get"
Song 1051: "Real Dope Boy"


...

Song 1: "09' Freestyle"
Song 2: "4 da Kidz"
Song 3: "4 da Kidz (Prime Day Show x Kid Cudi)"
Song 4: "50 Ways to Make a Record"
Song 5: "Adventures"
Song 6: "Afterwards (Bring Yo Friends)"
Song 7: "Alien (Demo)"
Song 8: "Alive (Nightmare)"
Song 9: "All Along"
Song 10: "All In"
Song 11: "AMEN"
Song 12: "Angel"
Song 13: "Angels & Demons"
Song 14: "Angered Kids"
Song 15: "Anomaly"
Song 16: "Another Day"
Song 17: "Arabic"
Song 18: "Ashinâ€™ Kusher"
Song 19: "Attention"
Song 20: "AT THE PARTY"
Song 21: "Awesomely Dope"
Song 22: "Balmain Jeans"
Song 23: "Baptized in Fire"
Song 24: "Beautiful Trip"
Song 25: "Beez"
Song 26: "Bet Ya Know*"
Song 27: "Bigger Than You (Do It Alone)"
Song 28: "BIRTHDAY MIX { FÃ¼ck My Ex }"
Song 29: "Brothers"
Song 30: "Brothers (Remix)"
Song 31: "Burn Baby Burn"
Song 32: "By Design"
Song 33: "Call Me Moon Man"
Song 34: "Can I Be"
Song 35: "Canâ€™t Believe It"
Song 36: "Canâ€™t Look In My Eyes"
Song 37: "Capcom"
Song 38: "Change (Outro)"
Song 39: "â€‹chilling by your

Song 258: "T.G.I.F."
Song 259: "That Girl"
Song 260: "The Adventures of Moon Man & Slim Shady"
Song 261: "The Biggest Grinder"
"The Chosen One" is not valid. Skipping.
Song 262: "The Commander"
Song 263: "The End"
Song 264: "The Flight of the Moon Man"
Song 265: "The Guide"
Song 266: "The Mood"
"The Movie" is not valid. Skipping.
Song 267: "The Nothing"
Song 268: "The Pale Moonlight"
Song 269: "The Prayer"
Song 270: "The Rage"
Song 271: "The Resurrection of Scott Mescudi"
Song 272: "The Return of Chip Douglas"
Song 273: "The Ruler and the Killer"
Song 274: "These Worries"
Song 275: "The Void"
Song 276: "Tim Westwood Freestyle"
Song 277: "To Da Top"
Song 278: "Too Bad I Have to Destroy You Now"
Song 279: "TORTURED"
Song 280: "To The Moon World Tour (Setlist)"
Song 281: "Trapped in My Mind"
Song 282: "Trauma"
Song 283: "Tree"
Song 284: "Troubled Boy"
Song 285: "Twitter Note (2015)"
Song 286: "Twitter Note 2 (2015)"
Song 287: "Twitter Note 3 (2015)"
Song 288: "Twitter Rant (September 2016

Song 1: "Act My Age"
Song 2: "A Cup of Coffee"
Song 3: "Agree To Disagree"
Song 4: "All You Need Is Love"
"Am I Wrong?" is not valid. Skipping.
"Apologize" is not valid. Skipping.
"Are You Tipping" is not valid. Skipping.
"Attention (Marshmellow Remix)" is not valid. Skipping.
"Attention (Zedd Solo Remix)" is not valid. Skipping.
Song 5: "Bad Photographs"
Song 6: "Believe"
Song 7: "Bigger Than Me"
Song 8: "Birds to the Ocean"
Song 9: "Birthday"
Song 10: "Birthday (Cash Cash Radio Mix)"
Song 11: "Birthday (Cash Cash Remix)"
Song 12: "Birthday (Le Youth Extended Remix)"
Song 13: "Birthday (Octava Disco House Radio Edit)"
Song 14: "Birthday (PRISMATIC WORLD TOUR VERSION)"
Song 15: "Black & Gold"
Song 16: "Black Widow (Demo)"
Song 17: "Bon AppÃ©tit"
Song 18: "Bon AppÃ©tit (3LAU Remix)"
Song 19: "Bon AppÃ©tit (Martin Jensen Remix)"
Song 20: "Bon AppÃ©tit (MUNA Remix)"
Song 21: "Bon AppÃ©tit (Solo Version)"
Song 22: "Boomerang"
"Born Yesterday" is not valid. Skipping.
Song 23: "Breakout"
Son

"OMG" is not valid. Skipping.
Song 178: "One of the Boys"
Song 179: "Only Love"
Song 180: "Part of Me"
Song 181: "Part of Me (Demo)"
"Part of Me (Instrumental)" is not valid. Skipping.
Song 182: "Part of Me (Jacques Lu Contâ€™s Thin White Duke Mix)"
Song 183: "Part of Me (Jacques Lu Contâ€™s Thin White Duke Radio Edit)"
Song 184: "Passenger"
"Peace At Last" is not valid. Skipping.
Song 185: "Peacock"
Song 186: "Peacock (Cory Enemy & Mia Moretti Vocal Club Mix)"
"Peacock (Demo)" is not valid. Skipping.
Song 187: "Pearl"
Song 188: "Pendulum"
"Perfect Mystery" is not valid. Skipping.
Song 189: "Perry Playland Theme"
Song 190: "Piercing"
Song 191: "Playing House"
Song 192: "Poor Unfortunate Souls"
Song 193: "Power"
Song 194: "Psycho Love"
"Purer The Gold" is not valid. Skipping.
Song 195: "Reborn"
"Remember Love" is not valid. Skipping.
Song 196: "Resilient"
"R.I.P." is not valid. Skipping.
Song 197: "Rise"
Song 198: "Rise (Monsieur Adi Radio Edit)"
Song 199: "Rise (Purity Ring Remix)"
Son

Song 1: "10AM/Save the World"
"10AM / Save the World (Instrumental)" is not valid. Skipping.
Song 2: "10 Freaky Girls"
"10 Freaky Girls (Instrumental)" is not valid. Skipping.
"1st Of Da Month" is not valid. Skipping.
"25K" is not valid. Skipping.
"All The Money (Instrumental - Bonus)" is not valid. Skipping.
"All The Way Live (Spider-Man: Across the Spider-Verse - Instrumental)" is not valid. Skipping.
"Am I Dreaming (Instrumental)" is not valid. Skipping.
"Annihilate (Spider-Man: Across the Spider-Verse - Instrumental)" is not valid. Skipping.
"Another Body" is not valid. Skipping.
Song 3: "Around Me"
Song 4: "Around Me (ChoppedNotSlopped)"
"Around Me (Instrumental)" is not valid. Skipping.
"BBL*" is not valid. Skipping.
"Big Mad" is not valid. Skipping.
"Bitch Ass" is not valid. Skipping.
"Bitch Boy*" is not valid. Skipping.
Song 5: "Blue Pill"
Song 6: "Blue Pill (Demo)"
Song 7: "Borrowed Love"
"Borrowed Love (Instrumental)" is not valid. Skipping.
"Bounce Back" is not valid. Skippi